In [1]:
import requests
from bs4 import BeautifulSoup
url = "https://www.opindia.com/latest-news/"
data = requests.get(url)

In [2]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd


In [4]:
data

<Response [200]>

In [5]:
soup = BeautifulSoup(data.content,"html.parser")


In [6]:
def fetch_article(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.content,"html.parser")
    articles = []
    for i in soup.find_all("h3",class_ = ["entry-title td-module-title"]):
        articles.append(i.find('a')['title'])     
    return articles

In [7]:
urllist = []
for i in range(2,21,1):
    url = "https://www.opindia.com/latest-news/page/" + str(i) + "/"
    urllist.append(url)


In [8]:
all_articles = []
for i in urllist:
    all_articles.extend(fetch_article(i))

In [9]:
p_art =[]
for i in all_articles:
    q = i.upper()
    import re
    q = re.sub("[^A-Z0-9 ]","",q)
    from nltk.stem import PorterStemmer
    tk_q = q.split(" ")
    sent = ""
    for j in tk_q:
        ps = PorterStemmer()
        sent = sent + " " + ps.stem(j).upper()
    p_art.append(sent)


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
A = tf.fit_transform(p_art).toarray()

In [11]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5)
cl_res = km.fit(A)

In [12]:
cl_res.labels_


array([0, 0, 3, ..., 1, 1, 1])

In [13]:
Q = pd.DataFrame(p_art,columns=["Article"])
Q['Cluster']=cl_res.labels_

In [18]:
Q.head(15)


,Article,Cluster,category
0,EXAMNESTI INDIA HEAD TRI TO PAINT GUJARATI CO...,0,geopolitics
1,IS IT TRUE SMRITI IRANI LASH OUT AT ARVIND KE...,0,geopolitics
2,TAMIL NADU BJP CHIEF K ANNAMALAI 5000 OTHER B...,3,sports
3,HYDERABAD OPEN CHARMINAR MOSQU FOR NAMAZ BHAG...,0,geopolitics
4,MAHARASHTRA NCP WORKER JOIN ISLAMIST TO OUTRA...,3,sports
5,EXAMNESTI INDIA HEAD TRI TO PAINT GUJARATI CO...,0,geopolitics
6,PANCHAYAT STRIKE A CHORD WITH THE INDIAN AUDI...,2,religion
7,BOLLYWOOD SUPERSTAR NEAR 60 PAIR AGAINST WOME...,2,religion
8,THREAT TO NUPUR SHARMA ONC AGAIN SHOW HOW IND...,3,sports
9,THE NEW YORK TIME OUTDO ITSELF IN STUPID SAY ...,4,entertainment


In [15]:
E = {1 : "politics",
2 : "religion",
3 : "sports",
4 : "entertainment",
0 : "geopolitics"}

In [16]:
R = []
for i in Q.Cluster:
    R.append(E[i])

Q['category'] = R

In [17]:
Q

,Article,Cluster,category
0,EXAMNESTI INDIA HEAD TRI TO PAINT GUJARATI CO...,0,geopolitics
1,IS IT TRUE SMRITI IRANI LASH OUT AT ARVIND KE...,0,geopolitics
2,TAMIL NADU BJP CHIEF K ANNAMALAI 5000 OTHER B...,3,sports
3,HYDERABAD OPEN CHARMINAR MOSQU FOR NAMAZ BHAG...,0,geopolitics
4,MAHARASHTRA NCP WORKER JOIN ISLAMIST TO OUTRA...,3,sports
...,...,...,...
1971,UTTAR PRADESH ACTION EXPECT AGAINST NISHAD P...,1,politics
1972,MAHARASHTRA POLIC INSPECTOR UNKNOWINGLI DISRE...,1,politics
1973,CONGRESS PARTI TARGET MODI GOVT MAKE SIX CLAI...,1,politics
1974,NCP SUPREMO SHARAD PAWAR SAY SEDIT LAW MUST G...,1,politics
